In [5]:
import pyquery as pq
from selenium import webdriver
import pandas as pd
import time
import os
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [6]:
driver = webdriver.Firefox()
driver.implicitly_wait(5)
driver.get("https://www.lagou.com/")

In [7]:
wait = WebDriverWait(driver,4)
button = wait.until(EC.element_to_be_clickable((By.XPATH,'.//*[@id="cboxClose"]')))
button.click()
print("点击了")

点击了


In [9]:
while True:
    i = input("已登陆成功请输入“OK”：")
    if i == 'OK':
        break
    elif i == 'quit':
        print("取消执行，关闭！")
        os._exit(1)

已登陆成功请输入“OK”：Ok
已登陆成功请输入“OK”：OK


In [10]:


def getPosition(items):
    datalist=[]
    for item in items.items():
        temp = dict()
        temp['职位ID']= item.attr('data-positionid')
        temp['职位名']= item.attr('data-positionname')
        temp['薪资范围']= item.attr('data-salary')
        temp['公司ID']= item.attr('data-companyid')
        temp['公司名']= item.attr('data-company')
        temp['职位链接']=pq.PyQuery(item).find(".position_link").attr("href")
        temp['发布时间']=pq.PyQuery(item).find(".format-time").text()
        temp['猎头名称']=pq.PyQuery(item).find(".hr_name").text()
        temp['猎头ID']=pq.PyQuery(item).find(".target_hr").text()
        temp['工作经验']=pq.PyQuery(item).find(".p_bot>.li_b_l").remove(".money").text()
        temp['公司主页']=pq.PyQuery(item).find(".company_name>a").attr('href')
        temp['公司描述']=pq.PyQuery(item).find(".industry").text()
        temp['岗位亮点']=pq.PyQuery(item).find(".li_b_r").text()

        datalist.append(temp)
    return datalist

In [15]:
print("开始执行采集")
data = []
driver.get("https://www.lagou.com/jobs/list_Php/p-city_0?px=new&gj=3年及以下#filterBox")
while True:
    but_class = driver.find_element_by_css_selector(".pager_next").get_attribute('class')

    if but_class == 'pager_next ':
        driver.find_element_by_xpath("//span[@action='next']").click()
        items = pq.PyQuery(driver.page_source).find(".con_list_item")
        data += getPosition(items)
        time.sleep(2)
    else:
        print('列表采集结束')
        break

开始执行采集
列表采集结束


In [16]:
csv = pd.DataFrame(data)
csv.to_csv("lagou2.csv")

In [17]:
import os
print(os.path.abspath('.'))

C:\Users\PC\Untitled Folder


In [19]:
urllist=csv[['职位链接','职位ID']].values.tolist()
import random
for row in urllist:
    driver.get(row[0])
    job_request =pq.PyQuery(driver.page_source).find(".job_request>h3").text()
    text =pq.PyQuery(driver.page_source).find(".job-detail").text()
    if "\u2200b" in text:
        print("我错了")
        print(text)
    else :
        with open("./data/quanguo_php_new.txt","a+",encoding='utf-8') as f:
            f.write(job_request+"\n"+text)
            f.write("\n========================================================================================================\n")
            time.sleep(2+random.randint(0,4))
print("职位采集完成")

职位采集完成
